In [1]:
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.tokenize import sent_tokenize, word_tokenize
from __future__ import print_function
import torch
from torch.autograd import Variable
import random
from sklearn.model_selection import train_test_split

In [2]:
from keras import backend as K
from keras.layers import Embedding,Dense
from keras.layers import LSTM, Input, merge, Lambda
from keras.layers.wrappers import Bidirectional
from keras.layers.convolutional import Convolution1D
from keras.models import Model
import numpy as np

Using TensorFlow backend.


In [3]:
file = "glove.6B.100d.txt"
import numpy as np
def loadGloveModel(gloveFile):
    print ("Loading Glove Model")
    
    with open(gloveFile, encoding="utf8" ) as f:
       content = f.readlines()
    model = {}
    for line in content:
        splitLine = line.split()
        word = splitLine[0]
        embedding = np.array([float(val) for val in splitLine[1:]])
        model[word] = embedding
    print ("Done.",len(model)," words loaded!")
    return model


In [4]:
model= loadGloveModel(file)   
stemmer = PorterStemmer()
lemmatiser= WordNetLemmatizer()


Loading Glove Model
Done. 400001  words loaded!


In [5]:
import string
questions=[]
answers=[]
answer=[]
length=30
sentence_length=length
with open("sample1.txt","r") as f:
    choice=0
    for line in f:
        line=line.lower()
        sentence=line.translate(string.punctuation)
        sentence=line[2:].split()
        word_vec=[]
        for word in sentence:
            if word in model:
                word_vec.append(model[word])
            else:
                stemmed_word=stemmer.stem(word)
                lemmatised_word=lemmatiser.lemmatize(word)
                if stemmed_word in model:
                    word_vec.append(model[stemmed_word])
                elif lemmatised_word in model:
                    word_vec.append(model[lemmatised_word])        
        if choice==0:
            choice=1
            if len(word_vec)<=sentence_length:
                for vec_len in range(length-len(word_vec)):
                    word_vec.append(model["0"])
            questions.append(word_vec[:length])
            
        else:
            choice=0
            if len(word_vec)<=sentence_length:
                for vec_len in range(length-len(word_vec)):
                    word_vec.append(model["0"])
                answers.append(word_vec)
                answer.append(line)
            else:
                del questions[-1]
                
        

In [6]:
input_vec_len=100
questions=np.asarray(questions)
answers=np.asarray(answers)

In [7]:
from sklearn.utils import shuffle

In [8]:
questions,answers=shuffle(questions,answers)

In [9]:
# questions=questions[1000:2000]
# answers=answers[1000:2000]

In [10]:
X_train, X_test, y_train, y_test = train_test_split(questions, answers, test_size=0.05, random_state=42)

In [11]:
bad_y_train=np.concatenate((y_test,y_train),axis=0)
bad_y_train=bad_y_train[:len(y_train)]

In [12]:
bad_y_train.shape,y_test.shape

((906, 30, 100), (48, 30, 100))

In [13]:
import keras
# from keras_self_attention import SeqSelfAttention

In [14]:
questions, good_answers, bad_answers=X_train,y_train,bad_y_train
Y = np.zeros(shape=(questions.shape[0],))

In [15]:
bad_answers.shape

(906, 30, 100)

In [16]:
def cosine_distance(vests):
    x, y = vests
    x = K.l2_normalize(x, axis=-1)
    y = K.l2_normalize(y, axis=-1)
    return -K.mean(x * y, axis=-1, keepdims=True)

def cos_dist_output_shape(shapes):
    shape1, shape2 = shapes
    return (shape1[0],1)

def loss_shape(shapes):
    shape1, shape2 = shapes
    return  0

In [17]:
import tensorflow as tf
from keras import backend as k
from keras.layers import Embedding,Dense,Flatten,Activation,RepeatVector,Permute,Multiply,Add
import keras
# from keras_self_attention import SeqSelfAttention

margin = 0.2
enc_timesteps = 60
dec_timesteps = 60
hidden_dim = 15
epoch=100
maxlen=60
# initialize the question and answer shapes and datatype
#                                       
question = Input(shape=(length,100))
answer = Input(shape=(length,100))
#                                                         
answer_good = Input(shape=(length,100))
answer_bad = Input(shape=(length,100))


# first hop
q1_bilstm = Bidirectional(LSTM(activation='tanh', units=hidden_dim, return_sequences=True))(question)
q1_atten = Dense(1, activation='tanh')(q1_bilstm)
q1_atten = Flatten()(q1_atten)
q1_atten = Activation('softmax')(q1_atten)
q1_atten = RepeatVector(length)(q1_atten)
q1_atten = Permute([2, 1])(q1_atten)
q1_sent_representation = Lambda(lambda xin: K.sum(xin[0]*xin[1], axis=-2), output_shape=(length,))([q1_bilstm, q1_atten])

a1_bilstm = Bidirectional(LSTM(activation='tanh', units=hidden_dim, return_sequences=True))(answer)
a1_atten = Dense(1, activation='tanh')(Add()([a1_bilstm,q1_bilstm]))
a1_atten = Flatten()(a1_atten)
a1_atten = Activation('softmax')(a1_atten)
a1_atten = RepeatVector(length)(a1_atten)
a1_atten = Permute([2, 1])(a1_atten)
a1_sent_representation = Lambda(lambda xin: K.sum(xin[0]*xin[1], axis=-2), output_shape=(length,))([a1_bilstm, a1_atten])

question_enc_1 = q1_sent_representation
answer_enc_1 = a1_sent_representation
distance1 = Lambda(cosine_distance, output_shape=cos_dist_output_shape)([question_enc_1, answer_enc_1])

# second hop
q2_bilstm=Bidirectional(LSTM(activation='tanh', units=hidden_dim, return_sequences=True))(question)
q2_atten=attention = Dense(1, activation='tanh')(Multiply()([q2_bilstm,q1_bilstm]))
q2_atten = Flatten()(q2_atten)
q2_atten = Activation('softmax')(q2_atten)
q2_atten = RepeatVector(length)(q2_atten)
q2_atten = Permute([2, 1])(q2_atten)
q2_sent_representation = Lambda(lambda xin: K.sum(xin[0]*xin[1], axis=-2), output_shape=(length,))([q2_bilstm, q2_atten])

a2_bilstm=Bidirectional(LSTM(activation='tanh', units=hidden_dim, return_sequences=True))(answer)
a2_atten=attention = Dense(1, activation='tanh')(Multiply()([a1_bilstm,Add()([a2_bilstm,q2_bilstm])]))
a2_atten = Flatten()(a2_atten)
a2_atten = Activation('softmax')(a2_atten)
a2_atten = RepeatVector(length)(a2_atten)
a2_atten = Permute([2, 1])(a2_atten)
a2_sent_representation = Lambda(lambda xin: K.sum(xin[0]*xin[1], axis=-2), output_shape=(length,))([a2_bilstm, a2_atten])

question_enc_2 = q2_sent_representation
answer_enc_2 = a2_sent_representation
distance2 = Lambda(cosine_distance, output_shape=cos_dist_output_shape)([question_enc_2, answer_enc_2])

# third hop
q3_bilstm=Bidirectional(LSTM(activation='tanh', units=hidden_dim, return_sequences=True))(question)
q3_atten=attention = Dense(1, activation='tanh')(Multiply()([q3_bilstm,q2_bilstm]))
q3_atten = Flatten()(q3_atten)
q3_atten = Activation('softmax')(q3_atten)
q3_atten = RepeatVector(length)(q3_atten)
q3_atten = Permute([2, 1])(q3_atten)
q3_sent_representation = Lambda(lambda xin: K.sum(xin[0]*xin[1], axis=-2), output_shape=(length,))([q3_bilstm, q3_atten])

a3_bilstm=Bidirectional(LSTM(activation='tanh', units=hidden_dim, return_sequences=True))(answer)
a3_atten=attention = Dense(1, activation='tanh')(Multiply()([a1_bilstm,Add()([a3_bilstm,q3_bilstm])]))
a3_atten = Flatten()(a3_atten)
a3_atten = Activation('softmax')(a3_atten)
a3_atten = RepeatVector(length)(a3_atten)
a3_atten = Permute([2, 1])(a3_atten)
a3_sent_representation = Lambda(lambda xin: K.sum(xin[0]*xin[1], axis=-2), output_shape=(length,))([a3_bilstm, a3_atten])

question_enc_3 = q3_sent_representation
answer_enc_3 = a3_sent_representation
distance3 = Lambda(cosine_distance, output_shape=cos_dist_output_shape)([question_enc_3, answer_enc_3])


distance=Lambda(lambda x: x[0] + x[1] + x[2],output_shape=lambda x: x[0])([distance1,distance2,distance3])

lstm_model = Model(inputs=[question, answer], outputs=[distance])
good_similarity = lstm_model([question, answer_good])

bad_similarity = lstm_model([question, answer_bad])

loss=Lambda(lambda x: K.relu(margin - x[0] + x[1]),output_shape=lambda x: x[0])([good_similarity, bad_similarity])


# return training and prediction model
training_model = Model(inputs=[question, answer_good, answer_bad], outputs=loss, name='training_model')
training_model.compile(loss=lambda y_true, y_pred: y_pred, optimizer="rmsprop")
prediction_model = Model(inputs=[question, answer_good], outputs=good_similarity, name='prediction_model')
prediction_model.compile(loss=lambda y_true, y_pred: y_pred, optimizer="rmsprop")

 

hi


In [42]:
for i in range(300):
    bad_answers=[y_train[random.randint(0,len(y_train)-1)] for i in range(len(y_train)) ]
    bad_answers=np.asarray(bad_answers)
    print(i)
    training_model.fit(
                [questions, good_answers, bad_answers],
                Y,
                epochs=1,
                batch_size=10,
                validation_split=0.05,
                verbose=1
            )


0
Train on 860 samples, validate on 46 samples
Epoch 1/1
120/860 [===>..........................] - ETA: 19s - loss: 0.0784

KeyboardInterrupt: 

In [43]:
acc=0
for i in range(len(X_test)):
    # pad the data and get it in desired format
    print("################################",i)
    answers=y_test
    question=[X_test[i]]*len(X_test)
#     question=[X_test[i]]*len(X_test)
    # get the similarity score
    sims = prediction_model.predict([question, answers])
    sims=list(sims)

    a=sorted(np.unique(sims),reverse=True)
    element=sims[i]
    acc+=1/(a.index(element)+1)

print("MRR",acc/len(X_test))    
#     print(i,len(sims))
    


################################ 0
1.0
################################ 1
1.2
################################ 2
1.2588235294117647
################################ 3
1.330252100840336
################################ 4
2.330252100840336
################################ 5
3.330252100840336
################################ 6
3.4135854341736693
################################ 7
3.4520469726352077
################################ 8
4.452046972635207
################################ 9
5.452046972635207
################################ 10
5.552046972635207
################################ 11
6.552046972635207
################################ 12
7.552046972635207
################################ 13
7.573323568379887
################################ 14
7.684434679490998
################################ 15
7.7511013461576646
################################ 16
8.751101346157665
################################ 17
9.751101346157665
################################ 18
9.893958489014807
########

In [ ]:
from keras.models import load_model

prediction_model.save('Multi hop interactive attention network model-b.h5')  # creates a HDF5 file 'my_model.h5'
# del model  # deletes the existing model

# returns a compiled model
# identical to the previous one
# model = load_model('my_model.h5')